# Intérroger des documents (RAG)

Commencez par télécharger le document qui va vous servir de base de connaissance.

In [1]:
# Download a file
import wget

file_path = "my_document.pdf"
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

Puis instancier la connexion à l'API Albert.

In [26]:
# Request configuration
import requests
import os

base_url = os.getenv("API_URL") # e.g. "http://localhost:8000/v1"
api_key = os.getenv("API_KEY")

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_. 

In [7]:
# Upload a file
collection = "leo"
model = "BAAI/bge-m3"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files.`

In [11]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")

response.json()

{'object': 'list',
 'data': [{'object': 'file',
   'id': 'aac6e368-736d-43a0-b579-30da45deec66',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1721408088}]}

In [12]:
file_id = response.json()["data"][0]["id"]

In [16]:
# Display tools parameters
response = session.get(f"{base_url}/tools")
for tool in response.json()["data"]:
    if tool["id"] == "BaseRAG":
        print(tool["description"].strip())

Base RAG, basic retrival augmented generation.

    Args:
        embeddings_model (str): OpenAI embeddings model
        collection (Optional[List[str]], optional): List of collections to search in. Defaults to None (all collections).
        file_ids (Optional[List[str]], optional): List of file IDs in the selected collections (after upload files). Defaults to None (all files are selected).
        k (int, optional): Top K per collection (max: 6). Defaults to 4.
        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.

    DEFAULT_PROMPT_TEMPLATE:
        "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}

Documents :

{files}"


In [27]:
# Chat completions
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Qui est Ulrich Tan ?"}],
    "stream": False,
    "n": 1,
    "tools": [
        {
            "function": {
                "name": "BaseRAG",
                "parameters": {
                    "embeddings_model": model,
                    "collections": [collection],
                    "k": 2,
                },
            },
            "type": "function",
        }
    ],
}

response = session.post(f"{base_url}/chat/completions", json=data)
print(response.json()["choices"][0]["message"]["content"])

Selon les documents, Ulrich Tan est le chef du pôle Datamin du département "Etablab".
